In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install sacremoses
!pip install git+https://github.com/nltk/nltk_contrib.git#egg=nltk_contrib
!pip install jiwer nltk sacrebleu rouge-score
!pip install --upgrade jiwer
!pip install torchmetrics
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 12.5 MB/s eta 0:00:00
  Cloning https://github.com/nltk/nltk_contrib.git to /tmp/pip-install-d78l8red/nltk-contrib_f382bc198e6740f4b951879c2de93899
  Running command git clone --filter=blob:none --quiet https://github.com/nltk/nltk_contrib.git /tmp/pip-install-d78l8red/nltk-contrib_f382bc198e6740f4b951879c2de93899
  Resolved https://github.com/nltk/nltk_contrib.git to commit 95d1806e2f4e89e960b76a685b1fba2eaa7d5142
  Preparing metadata (setup.py) ... done
  Created wheel for nltk_contrib: filename=nltk_contrib-3.8.1-py3-none-any.whl size=682136 sha256=26ba50eacd15fb9c7a761d731aa68f6a533854dd96bc94f0a47d31a8f7c16bb3
  Stored in directory: /tmp/pip-ephem-wheel-cache-dccep4xt/wheels/df/ab/e3/f99b22cdc83586c32ba851d962379a56a1e7f1bdb50aa41f7a
Successfully built nltk_contrib
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.4/106.4 kB 5.3 MB/s eta 0:00:0

In [ ]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import MBartForConditionalGeneration, MBartTokenizer, AdamW, AutoTokenizer
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
import sacrebleu
from nltk.translate.meteor_score import meteor_score
import jiwer
from jiwer import wer
from rouge_score import rouge_scorer
from torchmetrics.text import TranslationEditRate
from tqdm import tqdm

train

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/nlp_project/Dataset/phinc/train.csv")
val_df = pd.read_csv("/content/drive/MyDrive/nlp_project/Dataset/phinc/val.csv")

In [ ]:
class TranslationDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text = self.data['cs_query'][idx]
        target_text = self.data['en_query'][idx]

        input_ids = self.tokenizer(' '.join(str(input_text).split()[:50]), return_tensors='pt', truncation=True,max_length=50,padding='max_length')['input_ids'].squeeze()
        labels = self.tokenizer(' '.join(str(target_text).split()[:50]), return_tensors='pt', truncation=True,max_length=50,padding='max_length')['input_ids'].squeeze()

        return {"input_ids": input_ids, "labels": labels}

In [ ]:
# Define the translation dataset with references
class TranslationDatasetWithRefs(Dataset):
    def __init__(self, data):
        self.data = data
        self.tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text = self.data['cs_query'][idx]
        target_text = self.data['en_query'][idx]

        input_ids = self.tokenizer(' '.join(str(input_text).split()), return_tensors='pt',truncation=True,padding='max_length', max_length=50)['input_ids'].squeeze()
        labels = self.tokenizer(' '.join(str(target_text).split()), return_tensors='pt',truncation=True,padding='max_length', max_length=50)['input_ids'].squeeze()

        # Convert labels to text without special tokens
        references = [self.tokenizer.decode(labels, skip_special_tokens=True)]

        return {"input_ids": input_ids, "labels": labels, "references": references}

In [ ]:
# Create translation datasets and dataloaders for training and validation
train_dataset = TranslationDataset(train_df)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_dataset = TranslationDatasetWithRefs(val_df)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False)  # No need to shuffle validation data



```
# This is formatted as code
```

### mBERT

In [ ]:
# Set up the model and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt").to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [ ]:
# Fine-tuning loop with validation and model checkpointing
num_epochs = 5
best_bleu = 0.0

for epoch in range(num_epochs):
    total_loss = 0.0
    model.train()

    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs} (Training)"):
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}: Training Loss - {average_loss}")

    # Validation loop
    model.eval()
    bleu_scores = []

    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs} (Validation)"):
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)
            references = batch['references']

            # Generate predictions
            generated_ids = model.generate(input_ids, max_length=128)  # Adjust max_length as needed

            # Convert generated IDs to text
            predictions = [val_dataset.tokenizer.decode(generated_ids[0], skip_special_tokens=True)]

            # Calculate BLEU score
            bleu_score = corpus_bleu(references, predictions)
            bleu_scores.append(bleu_score)

    # Calculate average BLEU score
    avg_bleu = sum(bleu_scores) / len(bleu_scores)

    print(f"Epoch {epoch + 1}/{num_epochs}: Average BLEU - {avg_bleu}\n")

    # Save the model if the average BLEU score is the best so far
    if avg_bleu > best_bleu:
        best_bleu = avg_bleu
        model.save_pretrained(f"/content/drive/MyDrive/nlp_project/results/mBART-PHINC-WO-Pre/best_model_{epoch}_{avg_bleu}")
        print("best model saved!\n\n")

print("Training completed.")

Epoch 1/5 (Training): 100%|██████████| 2404/2404 [18:46<00:00,  2.13it/s]


Epoch 1/5: Training Loss - 1.3041881373192428


Epoch 1/5 (Validation):  12%|█▏        | 85/722 [01:07<08:24,  1.26it/s]/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
Epoch 1/5 (Validation): 100%|██████████| 722/722 [09:29<00:00,  1.27it/s]


Epoch 1/5: Average BLEU - 0.640257875178401

best model saved!




Epoch 2/5 (Training): 100%|██████████| 2404/2404 [18:48<00:00,  2.13it/s]


...

Epoch 4/5: Average BLEU - 0.6478491719236364



Epoch 5/5 (Training): 100%|██████████| 2404/2404 [18:46<00:00,  2.13it/s]


Epoch 5/5: Training Loss - 0.09520684312753994


Epoch 5/5 (Validation): 100%|██████████| 722/722 [09:16<00:00,  1.30it/s]

Epoch 5/5: Average BLEU - 0.6403348835962402

Training completed.


test

In [ ]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import MarianMTModel, MarianTokenizer, AdamW
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
import sacrebleu
from nltk.translate.meteor_score import meteor_score
import jiwer
from jiwer import wer
from rouge_score import rouge_scorer
from torchmetrics.text import TranslationEditRate
from tqdm import tqdm
import evaluate

In [ ]:
# Load the test data
test_df = pd.read_csv("/content/drive/MyDrive/nlp_project/Dataset/phinc/test.csv")

In [ ]:
# Create a translation dataset and dataloader for testing
test_dataset = TranslationDatasetWithRefs(test_df)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
# Load the best model
model = MBartForConditionalGeneration.from_pretrained("/content/drive/MyDrive/nlp_project/results/mBART-PHINC-WO-Pre/best_model_1_0.6626252564753994").to(device)

model.eval()
bleu_scores = []
wer_scores = []
ter_scores = []
meteor_scores = []
rouge1_scores = []
rouge2_scores = []
#ter = TranslationEditRate(normalize=True)


# Load the TER metric
ter = evaluate.load("ter")
wer = evaluate.load("wer")
rouge = evaluate.load('rouge')
bleu = evaluate.load("bleu")
with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Testing"):
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        references = batch['references']


        # Generate predictions
        generated_ids = model.generate(input_ids, max_length=100)  # Adjust max_length as needed

        # Convert generated IDs to text
        predictions = [test_dataset.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)]


        # Convert generated IDs to text
        #predictions = [val_dataset.tokenizer.decode(generated_ids[0], skip_special_tokens=True)]

        bleu_score = bleu.compute(predictions=predictions[0], references=references[0])
        bleu_scores.append(bleu_score['bleu'])


        # Calculate WER score
        wer_score = wer.compute(predictions=predictions[0],references=references[0])
        wer_scores.append(wer_score)

        # Calculate TER score
        ter_score = ter.compute(predictions=predictions[0],references=references[0],
                       case_sensitive=False)
        #ter_score = TER(references[0][0], predictions[0])
        ter_scores.append(ter_score['score'])

        # Calculate ROUGE score
        rouge_score = rouge.compute(predictions=predictions[0],references=references[0])
        rouge1_scores.append(rouge_score['rouge1'])
        rouge2_scores.append(rouge_score['rouge2'])

Testing: 100%|██████████| 1237/1237 [14:34<00:00,  1.41it/s]


In [ ]:
# Calculate average scores
avg_bleu = sum(bleu_scores) / len(bleu_scores)
avg_wer = sum(wer_scores) / len(wer_scores)
avg_ter = sum(ter_scores) / len(ter_scores)
avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores)

# Print metric values
print("Testing Scores")
print("Metric\t\t\tWER\t\t\tTER\t\t\tBLEU\t\t\tROUGE-1\t\t\tROUGE-2")
print(f"Averages\t\t{avg_wer:.4f}\t\t\t{avg_ter/100:.4f}\t\t\t{avg_bleu:.4f}\t\t\t{avg_rouge1:.4f}\t\t\t{avg_rouge2:.4f}")

Testing Scores
Metric			WER			TER			BLEU			ROUGE-1			ROUGE-2
Averages		0.9092			0.8539			0.1450			0.5458			0.3139


#MarianMT_finetune

In [ ]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import MarianMTModel, MarianTokenizer, AdamW,AutoTokenizer
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
import sacrebleu
from nltk.translate.meteor_score import meteor_score
import jiwer
from jiwer import wer
from rouge_score import rouge_scorer
from torchmetrics.text import TranslationEditRate
from tqdm import tqdm
import evaluate
from transformers import AutoModelForSeq2SeqLM

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/nlp_project/Dataset/phinc/train.csv")
val_df = pd.read_csv("/content/drive/MyDrive/nlp_project/Dataset/phinc/val.csv")

In [ ]:
class TranslationDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-cs")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text = self.data['cs_query'][idx]
        target_text = self.data['en_query'][idx]

        input_ids = self.tokenizer(str(input_text), return_tensors='pt', truncation=True,max_length=100,padding='max_length')['input_ids'].squeeze()
        labels = self.tokenizer(str(target_text), return_tensors='pt', truncation=True,max_length=100,padding='max_length')['input_ids'].squeeze()

        return {"input_ids": input_ids, "labels": labels}

In [ ]:
# Define the translation dataset with references
class TranslationDatasetWithRefs(Dataset):
    def __init__(self, data):
        self.data = data
        self.tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-cs")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text = self.data['cs_query'][idx]
        target_text = self.data['en_query'][idx]

        input_ids = self.tokenizer(str(input_text), return_tensors='pt',truncation=True,padding='max_length', max_length=100)['input_ids'].squeeze()
        labels = self.tokenizer(str(target_text), return_tensors='pt',truncation=True,padding='max_length', max_length=100)['input_ids'].squeeze()

        # Convert labels to text without special tokens
        references = [self.tokenizer.decode(labels, skip_special_tokens=True)]

        return {"input_ids": input_ids, "labels": labels, "references": references}

In [ ]:
# Create translation datasets and dataloaders for training and validation
train_dataset = TranslationDataset(train_df)
train_dataloader = DataLoader(train_dataset, batch_size=50, shuffle=True)
val_dataset = TranslationDatasetWithRefs(val_df)
val_dataloader = DataLoader(val_dataset, batch_size=50, shuffle=False)  # No need to shuffle validation data

In [ ]:
# Set up the model and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MarianMTModel.from_pretrained("/content/drive/MyDrive/nlp_project/results/MarianMT/best_model").to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Fine-tuning loop with validation and model checkpointing
num_epochs = 3
best_loss = 9999

for epoch in range(num_epochs):
    total_loss = 0.0
    model.train()

    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs} (Training)"):
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}: Training Loss - {average_loss}")

    if average_loss < best_loss:
        best_loss = average_loss
        model.save_pretrained("/content/drive/MyDrive/nlp_project/results/MarianMT_finetune/best_model_finetune")
        print("best model saved!\n\n")

print("Training completed.")

Epoch 1/3 (Training): 100%|██████████| 193/193 [02:38<00:00,  1.22it/s]


Epoch 1/3: Training Loss - 0.3803178893469776
best model saved!




Epoch 2/3 (Training): 100%|██████████| 193/193 [02:36<00:00,  1.23it/s]


Epoch 2/3: Training Loss - 0.3810721668245879


Epoch 3/3 (Training): 100%|██████████| 193/193 [02:35<00:00,  1.24it/s]

Epoch 3/3: Training Loss - 0.3808857140763436
Training completed.


#test

In [ ]:
# Load the test data
test_df = pd.read_csv("/content/drive/MyDrive/nlp_project/Dataset/phinc/test.csv")

In [ ]:
# Create a translation dataset and dataloader for testing
test_dataset = TranslationDatasetWithRefs(test_df)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
# Load the best model
model = MarianMTModel.from_pretrained("/content/drive/MyDrive/nlp_project/results/MarianMT_finetune/best_model_finetune").to(device)

model.eval()
bleu_scores = []
wer_scores = []
ter_scores = []
meteor_scores = []
rouge1_scores = []
rouge2_scores = []
#ter = TranslationEditRate(normalize=True)


# Load the TER metric
ter = evaluate.load("ter")
wer = evaluate.load("wer")
rouge = evaluate.load('rouge')
bleu = evaluate.load("bleu")
with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Testing"):
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        references = batch['references']


        # Generate predictions
        generated_ids = model.generate(input_ids, max_length=100)  # Adjust max_length as needed

        # Convert generated IDs to text
        predictions = [test_dataset.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)]


        # Convert generated IDs to text
        #predictions = [val_dataset.tokenizer.decode(generated_ids[0], skip_special_tokens=True)]

        bleu_score = bleu.compute(predictions=predictions[0], references=references[0])
        bleu_scores.append(bleu_score['bleu'])


        # Calculate WER score
        wer_score = wer.compute(predictions=predictions[0],references=references[0])
        wer_scores.append(wer_score)

        # Calculate TER score
        ter_score = ter.compute(predictions=predictions[0],references=references[0],
                       case_sensitive=False)
        #ter_score = TER(references[0][0], predictions[0])
        ter_scores.append(ter_score['score'])

        # Calculate ROUGE score
        rouge_score = rouge.compute(predictions=predictions[0],references=references[0])
        rouge1_scores.append(rouge_score['rouge1'])
        rouge2_scores.append(rouge_score['rouge2'])

Testing: 100%|██████████| 1237/1237 [08:38<00:00,  2.38it/s]


In [ ]:
# Calculate average scores
avg_bleu = sum(bleu_scores) / len(bleu_scores)
avg_wer = sum(wer_scores) / len(wer_scores)
avg_ter = sum(ter_scores) / len(ter_scores)
avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores)

# Print metric values
print("Testing Scores")
print("Metric\t\t\tWER\t\t\tTER\t\t\tBLEU\t\t\tROUGE-1\t\t\tROUGE-2")
print(f"Averages\t\t{avg_wer:.4f}\t\t\t{avg_ter/100:.4f}\t\t\t{avg_bleu:.4f}\t\t\t{avg_rouge1:.4f}\t\t\t{avg_rouge2:.4f}")

Testing Scores
Metric			WER			TER			BLEU			ROUGE-1			ROUGE-2
Averages		1.0047			0.9481			0.1017			0.4822			0.2501
